The following jupyter notebook demonstrates how a point file containing elk locations can be explored using an interactive map.

In [1]:
import folium # to make interactive maps
from folium.plugins import MarkerCluster, Draw # to group points on map and draw shapes
import geopandas as gpd # to work with geospatial data -- https://geopandas.org/en/stable/docs

In [2]:
# Load the data
location_file="data/ElkWinter08_09.csv"
location_df=gpd.read_file(location_file)

In [3]:
# Assign the coordinate reference system
location_df.crs = 'epsg:4326'

In [4]:
# View the data
location_df

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,geometry
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,None
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,None
2,146816,856,11/7/2008,0,245,40.415933,-105.646525,None
3,68067,881,11/7/2008,0,215,40.410008,-105.644883,None
4,146812,856,11/7/2008,0,305,40.412341,-105.644525,None
...,...,...,...,...,...,...,...,...
10480,146823,856,11/8/2008,0,65,40.388416,-105.522908,None
10481,146821,856,11/8/2008,2.3,220,40.388241,-105.5224,None
10482,146822,856,11/8/2008,0,155,40.387675,-105.521108,None
10483,68072,881,11/8/2008,0,315,40.384808,-105.518541,None


In [5]:
# Create geometry values
location_df = gpd.GeoDataFrame(
    location_df, geometry=gpd.points_from_xy(location_df.x, location_df.y)
)

In [6]:
# View the data again
location_df

,OBJECTID,CollarID,Date_MMDDY,Speed_mph,Heading,y,x,geometry
0,146467,843COW,3/5/2009,9.2,280,40.367308,-105.664583,POINT (-105.66458 40.36731)
1,146378,843COW,11/16/2008,1.2,345,40.358716,-105.656541,POINT (-105.65654 40.35872)
2,146816,856,11/7/2008,0,245,40.415933,-105.646525,POINT (-105.64652 40.41593)
3,68067,881,11/7/2008,0,215,40.410008,-105.644883,POINT (-105.64488 40.41001)
4,146812,856,11/7/2008,0,305,40.412341,-105.644525,POINT (-105.64453 40.41234)
...,...,...,...,...,...,...,...,...
10480,146823,856,11/8/2008,0,65,40.388416,-105.522908,POINT (-105.52291 40.38842)
10481,146821,856,11/8/2008,2.3,220,40.388241,-105.5224,POINT (-105.52240 40.38824)
10482,146822,856,11/8/2008,0,155,40.387675,-105.521108,POINT (-105.52111 40.38768)
10483,68072,881,11/8/2008,0,315,40.384808,-105.518541,POINT (-105.51854 40.38481)


In [7]:
# Create map and show the data
# Get Rocky Mountain National Park coordinates - from https://www.google.com/maps
m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=5)

# Create and add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Create a list of location coordinates
geo_df_list = [[location.xy[1][0], location.xy[0][0]] for location in location_df.geometry]

# Create a marker for each location and add them to the cluster
for location in geo_df_list:
    marker = folium.Marker(location=location)
    marker_cluster.add_child(marker)

# Create and add a drawing control to the map
draw = Draw(export=True)
draw.add_to(m)

# show the map
m

# Beacuse there appears to be a lot of test data, draw a rectangle to be used as an exclusion zone
# Click the rectangle and copy the geojson into the next code cell 

In [8]:
# Load in shaply shape and polygon packages to convert our geojson to a polygon
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

# Store the geojson from earlier and convert to a polygon
polygon_geojson={"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[-105.568657,40.357664],[-105.568657,40.369436],[-105.551834,40.369436],[-105.551834,40.357664],[-105.568657,40.357664]]]}}
polygon: Polygon = shape(polygon_geojson["geometry"])

# Exclude the data from within the polygon, storing it as a new filtered geodataframe
filtered_locations = location_df[~location_df.within(polygon)]

# Compare the number of locations before and after the filter
print(len(location_df))
len(filtered_locations)

10485


194

In [9]:
# Create a map of the filtered locations

m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=11)

filtered_locations.explore(m=m, marker_type="marker")

m

In [10]:
# Explore the data
filtered_locations.to_csv("data/filtered_elk_locations.csv")

## Extra

In [11]:
# Get the number of unique tracking devices
unique_col="CollarID"
filtered_locations[unique_col].unique()

array(['843COW', '856', '881'], dtype=object)

In [12]:
# Create a map using different colors for each device
ids =['843COW', '856', '881']
colors =['blue','green','red']

m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)

for index, row in filtered_locations.iterrows():
    folium.CircleMarker(location=(row['y'],row['x']),
                        color=colors[ids.index(row[unique_col])],
                        radius=5,
                        fill=True,
                        popup=row[unique_col]
                        ).add_to(m)
m


In [13]:
# Separate one device
animal_locations=filtered_locations[filtered_locations[unique_col]=='843COW']
len(animal_locations)

168

In [14]:
# Animate the locations of a specific device over time

import datetime

date_col="Date_MMDDY"

animal_locations_geojson={"features": []}
for index, row in animal_locations.iterrows():
    date1 = datetime.datetime.strptime(row[date_col], "%m/%d/%Y")
    date2 = date1#+datetime.timedelta(days=1)
    animal_locations_geojson["features"].append({"type":"Feature",
                                                 "properties":{"times": [date1.strftime("%Y-%m-%d"),date2.strftime("%Y-%m-%d")]},
                                   
                                     "geometry":{"type":"Point","coordinates":[row['x'],row['y']]}})
m = folium.Map(location=[40.384733209948294,-105.68988068298488,], zoom_start=11)
# folium.GeoJson(animal_locations_geojson).add_to(m)

folium.plugins.TimestampedGeoJson(
    animal_locations_geojson,
    date_options="YYYY-MM-DD",
    period="P1D",
    duration="P1D",
    add_last_point=False,
).add_to(m)

m